In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com"

# Replace with a valid key
training_key = "5f5792df332e4181a4b94727b10320dc"
prediction_key = "4a0af87d0448450b883baa586f0ee14e"
prediction_resource_id = "/subscriptions/b76cac46-9d96-470e-9488-3afb775e4767/resourceGroups/test/providers/Microsoft.CognitiveServices/accounts/test_prediction"

publish_iteration_name = "classifyModel"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

# Create a new project
print ("Creating project...")
project = trainer.create_project("My New Project")

Creating project...


In [2]:
grapfruit_tag = trainer.create_tag(project.id, "Grapefruit")
lemon_tag = trainer.create_tag(project.id, "Lemon")

In [6]:
base_image_url = "C:\\Users\\atorsins\\Desktop\\Community of practice\\Cognitive services\\data\\fruits\\"

print("Adding images...")

image_list = []

for image_num in range(0, 10):
    file_name = f"grapefruit_{image_num}.jpg"
    with open(base_image_url + "grapefruit\\" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[grapfruit_tag.id]))

for image_num in range(0, 10):
    file_name = f"lemon_{image_num}.jpg"
    with open(base_image_url + "lemon\\" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[lemon_tag.id]))

upload_result = trainer.create_images_from_files(project.id, images=image_list)
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


In [7]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


In [9]:
# Now there is a trained endpoint that can be used to make a prediction
predictor = CustomVisionPredictionClient(prediction_key, endpoint=ENDPOINT)

with open(base_image_url + "test_lemon.jpg", "rb") as image_contents:
    results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Lemon: 99.94%
	Grapefruit: 0.04%
